<a href="https://colab.research.google.com/github/lukasoares/Model_to-predict_stroke/blob/main/DataStrokeWithPysparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Iniciando o Spark no Google Colab:

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Desafio modulo 3 Spark") \
    .getOrCreate()

##Importando a base de dados

In [5]:
data= spark.read.csv('/content/drive/MyDrive/base_de_dados/Stroke-data/healthcare-dataset-stroke-data.csv', header = True, inferSchema = True)

In [6]:
data.show(5)

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

Como podemos ver, essa base de dados contém os tipos das colunas já categorizados de maneira apropriada, com excessão da categória "bmi". Foi feita a transformação dessa coluna para "DoubleType"

In [7]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [8]:
from pyspark.sql.types import DoubleType, StringType

In [9]:
data = data.withColumn('bmi', data['bmi'].cast(DoubleType()))
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [10]:
data.count()

5110

In [11]:
len(data.columns)

12

##Limpeza dos dados

In [12]:
from pyspark.sql import functions as f

Primeiro vamos tentar localizar os dados nulos de todas as colunas, utilizando um loop que soma 1 (ou True) se a coluna contém um dado nulo ou N/A.

In [13]:
data.select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in data.columns]).show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|bmi|smoking_status|stroke|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|201|             0|     0|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+



Foram localizados 201 dados nulos na coluna "bmi". Para a subtituição dessa valores foi utilizado a mediana. Para facilitar o processo, utilizei a função toPandas() para conseguir a mediana de maneira mais fácil.

In [14]:
data = data.withColumn("bmi", f.when(f.isnull('bmi'), data.toPandas().bmi.median()).otherwise(data["bmi"]))

In [15]:
data.show(5)

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21|28.1|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

"Com a classe "createOrReplaceTempView", é possível criar uma "view" no pyspark.sql, o que permite fazer consultas utilizando a linguagem SQL."

In [16]:
data.createOrReplaceTempView("dataView")

Após uma observação mais atenciosa, foi localizado 1544 dados "Unknown" da coluna "smoking_status". Como se trata de uma "coluna" muito relevante para a nossa análise, foi escolhido a exclusão desse dados.

In [17]:
for c in ["gender", "ever_married",  "work_type", "Residence_type", "smoking_status"]:
  spark.sql(f"SELECT DISTINCT({c}) FROM dataView").show()

+------+
|gender|
+------+
|Female|
| Other|
|  Male|
+------+

+------------+
|ever_married|
+------------+
|          No|
|         Yes|
+------------+

+-------------+
|    work_type|
+-------------+
| Never_worked|
|Self-employed|
|      Private|
|     children|
|     Govt_job|
+-------------+

+--------------+
|Residence_type|
+--------------+
|         Urban|
|         Rural|
+--------------+

+---------------+
| smoking_status|
+---------------+
|         smokes|
|        Unknown|
|   never smoked|
|formerly smoked|
+---------------+



In [18]:
data_modi = spark.sql('SELECT * from dataView WHERE smoking_status != "Unknown"')
data_modi.createOrReplaceTempView("dataView")

In [19]:
data_modi.createOrReplaceTempView("dataView")

In [20]:
data_modi.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21|28.1|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

#Análise dos Dados

##Análise por Gêneros
Inicialmente, realizou-se a análise da ocorrência de ataques cardíacos separando os indivíduos por gênero. Foram avaliados os dados das pessoas que já sofreram ataque cardíaco e os que nunca sofreram, para que fosse possível identificar possíveis diferenças entre os gêneros.

In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [22]:
df = data_modi.toPandas()

In [23]:
stroke_gender_1 = spark.sql('SELECT Gender, COUNT(stroke) FROM dataView WHERE STROKE == 1 group by gender').toPandas()

In [24]:
stroke_gender_2 = spark.sql('SELECT Gender, COUNT(stroke) FROM dataView WHERE STROKE == 0 group by gender').toPandas()
stroke_gender_2

,Gender,count(stroke)
0,Female,2042
1,Other,1
2,Male,1321


In [27]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.1, subplot_titles = ["Had a heart attack", "Did not have a heart attack"])
fig.add_trace(go.Bar(x=stroke_gender_1['count(stroke)'], y=stroke_gender_1['Gender'], orientation='h', name="Ataque Cardíaco"), row=1, col=1)
fig.add_trace(go.Bar(x=stroke_gender_2['count(stroke)'], y=stroke_gender_2['Gender'], orientation='h'), row=1, col=2)
fig.update_layout(yaxis_title='Gênero', height=400, width=800, template="plotly_dark", showlegend=False, title={'text': "Count of People", 'x': 0.5, 'y': 0.1, 'xanchor': 'center', 'yanchor': 'bottom'}, margin=dict(t=50))
fig.show()
